In [1]:
import numpy as np
import pandas as pd

#读取文件
parser = lambda date: pd.to_datetime(date, format='%d.%m.%Y')
train = pd.read_csv('/Users/ynshan/Desktop/predict/sales_train.csv', parse_dates=['date'],
                    date_parser=parser)
test = pd.read_csv('/Users/ynshan/Desktop/predict/test.csv')
submission = pd.read_csv('/Users/ynshan/Desktop/predict/sample_submission.csv')
items = pd.read_csv('/Users/ynshan/Desktop/predict/items.csv')
item_cats = pd.read_csv('/Users/ynshan/Desktop/predict/item_categories.csv')
shops = pd.read_csv('/Users/ynshan/Desktop/predict/shops.csv')

In [2]:

#查看信息
print('train:', train.shape, 'test:', test.shape, 'items:', items.shape, 'shops:', shops.shape,'cats:',item_cats.shape)


train: (2935849, 6) test: (214200, 3) items: (22170, 3) shops: (60, 2) cats: (84, 2)


In [3]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [4]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [6]:
item_cats.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [7]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [8]:
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month

train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month
0,2013-01-02,0,59,22154,999.00,1.0,2013,1
1,2013-01-03,0,25,2552,899.00,1.0,2013,1
2,2013-01-05,0,25,2552,899.00,-1.0,2013,1
3,2013-01-06,0,25,2554,1709.05,1.0,2013,1
4,2013-01-15,0,25,2555,1099.00,1.0,2013,1


In [9]:
#一共34个月份
train['date_block_num'].max()

33

In [10]:
# group by 
train_grp = train.groupby(['date_block_num','shop_id','item_id'])

In [11]:
# price mean by month
train_price = pd.DataFrame(train_grp.mean()['item_price']).reset_index()
train_price.head()

,date_block_num,shop_id,item_id,item_price
0,0,0,32,221.0
1,0,0,33,347.0
2,0,0,35,247.0
3,0,0,43,221.0
4,0,0,51,128.5


In [12]:
# count summary by month
train_monthly = pd.DataFrame(train_grp.sum()['item_cnt_day']).reset_index()
train_monthly.rename(columns={'item_cnt_day':'item_cnt'}, inplace=True)
train_monthly.head()

,date_block_num,shop_id,item_id,item_cnt
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [13]:
test_only = test[~test['item_id'].isin(train['item_id'].unique())]['item_id'].unique()
len(test_only)

363

In [14]:
# By shop
grp = train_monthly.groupby(['shop_id', 'item_id'])
train_shop = grp.agg({'item_cnt':['mean','sum','median','min','max']}).reset_index()
train_shop.columns = ['shop_id','item_id','cnt_mean_shop','cnt_sum_shop','cnt_med_shop','cnt_min_shop','cnt_max_shop']
train_shop.head()

,shop_id,item_id,cnt_mean_shop,cnt_sum_shop,cnt_med_shop,cnt_min_shop,cnt_max_shop
0,0,30,31.0,31.0,31.0,31.0,31.0
1,0,31,11.0,11.0,11.0,11.0,11.0
2,0,32,8.0,16.0,8.0,6.0,10.0
3,0,33,3.0,6.0,3.0,3.0,3.0
4,0,35,7.5,15.0,7.5,1.0,14.0


In [15]:
# By shop&category
train_cat_monthly = pd.merge(train_monthly, items, on=['item_id'], how='left')
grp = train_cat_monthly.groupby(['shop_id', 'item_category_id'])
train_shop_cat = grp.agg({'item_cnt':['sum','median']}).reset_index()
train_shop_cat.columns = ['shop_id','item_category_id','cnt_sum_cat_shop','cnt_med_cat_shop']
train_shop_cat.head()

,shop_id,item_category_id,cnt_sum_cat_shop,cnt_med_cat_shop
0,0,0,1.0,1.0
1,0,1,1.0,1.0
2,0,2,105.0,2.0
3,0,3,52.0,26.0
4,0,4,38.0,2.0


In [16]:
# Last month
train_last = train_monthly[train_monthly['date_block_num'] == 33]
train_last = train_last.drop(['date_block_num'], axis=1).rename(columns={'item_cnt':'cnt_sum_last'})
train_last.head()

,shop_id,item_id,cnt_sum_last
1577593,2,31,1.0
1577594,2,486,3.0
1577595,2,787,1.0
1577596,2,794,1.0
1577597,2,968,1.0


In [17]:
# Prev month
train_prev = train_monthly.copy()
train_prev['date_block_num'] = train_prev['date_block_num'] + 1
train_prev = train_prev.rename(columns={'item_cnt':'cnt_sum_prev'})
train_prev.head()

,date_block_num,shop_id,item_id,cnt_sum_prev
0,1,0,32,6.0
1,1,0,33,3.0
2,1,0,35,1.0
3,1,0,43,1.0
4,1,0,51,2.0


In [18]:
# Prev month by category
train_cat_prev = pd.merge(train_prev, items, on=['item_id'], how='left')
grp = train_cat_prev.groupby(['date_block_num','item_category_id'])
train_cat_prev = grp['cnt_sum_prev'].sum().reset_index()
train_cat_prev = train_cat_prev.rename(columns={'cnt_sum_prev':'cnt_sum_cat_prev'})
train_cat_prev.head()

,date_block_num,item_category_id,cnt_sum_cat_prev
0,1,0,1.0
1,1,1,1.0
2,1,2,1390.0
3,1,3,440.0
4,1,4,251.0


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
item_cats['item_category_name_len'] = item_cats['item_category_name'].apply(len)
item_cats['item_category_name_wc'] = item_cats['item_category_name'].apply(lambda x: len(str(x).split(' ')))

feature_cnt = 11
tfidf = TfidfVectorizer(max_df=0.6, max_features=feature_cnt, ngram_range=(1, 2))
txtFeatures = pd.DataFrame(tfidf.fit_transform(item_cats['item_category_name']).toarray())
cols = txtFeatures.columns
for i in range(feature_cnt):
    item_cats['item_category_name_tfidf_' + str(i)] = txtFeatures[cols[i]]

In [22]:
#txtFeatures

In [23]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_df=0.6, max_features=feature_cnt, ngram_range=(1, 2))
txtFeatures = pd.DataFrame(tfidf.fit_transform(items['item_name']).toarray())
cols = txtFeatures.columns
for i in range(feature_cnt):
    items['item_name_tfidf_' + str(i)] = txtFeatures[cols[i]]

In [24]:
shops['shop_name_len'] = shops['shop_name'].apply(len)
shops['shop_name_wc'] = shops['shop_name'].apply(lambda x: len(str(x).split(' ')))

shops['city'] = shops.shop_name.apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0])
shops['city'] = pd.Categorical(shops['city']).codes

In [25]:
#shops

In [26]:
def margeFeature(source): 
  d = source
  d = pd.merge(d, items, on=['item_id'], how='left')
  d = pd.merge(d, item_cats, on=['item_category_id'], how='left')
  d = pd.merge(d, shops, on=['shop_id'], how='left')

  d = pd.merge(d, train_shop, on=['shop_id','item_id'], how='left')
  d = pd.merge(d, train_shop_cat, on=['shop_id','item_category_id'], how='left')
  d = pd.merge(d, train_last, on=['shop_id','item_id'], how='left')
  #d = pd.merge(d, train_piv, on=['shop_id','item_id'], how='left')
  #d = pd.merge(d, train_prev, on=['date_block_num','shop_id','item_id'], how='left')
  #d = pd.merge(d, train_cat_prev, on=['date_block_num','item_category_id'], how='left')
  d = pd.merge(d, train_price, on=['date_block_num','shop_id','item_id'], how='left')
  
  #d.drop(['shop_id','shop_name','item_id','item_name','item_category_name'], axis=1, inplace=True)
  d.drop(['shop_name','item_name','item_category_name'], axis=1, inplace=True)
  d.fillna(0.0, inplace=True)
  return d

In [27]:
train_set = margeFeature(train_monthly)

X_train = train_set.drop(['item_cnt'], axis=1)
Y_train = train_set['item_cnt']

X_train.head()

,date_block_num,shop_id,item_id,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,...,city,cnt_mean_shop,cnt_sum_shop,cnt_med_shop,cnt_min_shop,cnt_max_shop,cnt_sum_cat_shop,cnt_med_cat_shop,cnt_sum_last,item_price
0,0,0,32,40,0.0,0.0,0.0,0.000000,0.0,0.0,...,29,8.0,16.0,8.0,6.0,10.0,3768.0,1.0,0.0,221.0
1,0,0,33,37,0.0,0.0,1.0,0.000000,0.0,0.0,...,29,3.0,6.0,3.0,3.0,3.0,617.0,1.0,0.0,347.0
2,0,0,35,40,0.0,0.0,0.0,0.000000,0.0,0.0,...,29,7.5,15.0,7.5,1.0,14.0,3768.0,1.0,0.0,247.0
3,0,0,43,40,0.0,0.0,0.0,0.000000,0.0,0.0,...,29,1.0,1.0,1.0,1.0,1.0,3768.0,1.0,0.0,221.0
4,0,0,51,57,0.0,0.0,0.0,0.561001,0.0,0.0,...,29,2.5,5.0,2.5,2.0,3.0,155.0,1.0,0.0,128.5


In [28]:
test['date_block_num'] = 34

X_test = margeFeature(test.drop(['ID'], axis=1))
X_test.head()

,shop_id,item_id,date_block_num,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,item_name_tfidf_4,item_name_tfidf_5,...,city,cnt_mean_shop,cnt_sum_shop,cnt_med_shop,cnt_min_shop,cnt_max_shop,cnt_sum_cat_shop,cnt_med_cat_shop,cnt_sum_last,item_price
0,5,5037,34,19,0.0,0.000000,0.0,0.0,0.0,0.0,...,3,1.444444,13.0,1.0,1.0,3.0,3217.0,1.0,0.0,0.0
1,5,5320,34,55,0.0,0.000000,0.0,0.0,0.0,0.0,...,3,0.000000,0.0,0.0,0.0,0.0,4884.0,1.0,0.0,0.0
2,5,5233,34,19,0.0,0.000000,0.0,0.0,0.0,0.0,...,3,2.000000,10.0,2.0,1.0,3.0,3217.0,1.0,1.0,0.0
3,5,5232,34,23,0.0,0.526615,0.0,0.0,0.0,0.0,...,3,1.000000,1.0,1.0,1.0,1.0,2016.0,1.0,0.0,0.0
4,5,5268,34,20,0.0,0.000000,0.0,0.0,0.0,0.0,...,3,0.000000,0.0,0.0,0.0,0.0,1583.0,1.0,0.0,0.0


In [29]:
#test

In [30]:
from sklearn import ensemble, metrics
from sklearn import linear_model, preprocessing

In [31]:
corr = train_set.corr()['item_cnt']
corr_imp_20 = corr.abs().sort_values(ascending=False)[1:20]
corr_imp_20

cnt_mean_shop                  0.762254
cnt_med_shop                   0.725460
cnt_sum_shop                   0.706630
cnt_med_cat_shop               0.663532
cnt_max_shop                   0.615076
cnt_sum_last                   0.309672
cnt_min_shop                   0.301173
item_name_tfidf_0              0.118692
item_category_name_tfidf_4     0.057616
item_name_tfidf_10             0.052097
item_name_tfidf_20             0.051680
item_category_name_tfidf_7     0.048432
item_name_tfidf_15             0.046391
item_name_tfidf_19             0.044255
item_category_name_wc          0.042112
item_category_name_tfidf_10    0.038810
item_name_tfidf_24             0.036616
item_name_tfidf_23             0.035621
item_name_tfidf_18             0.032376
Name: item_cnt, dtype: float64

In [32]:
import xgboost as xgb

sc = preprocessing.StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

reg = xgb.XGBRegressor(n_estimators=25, max_depth=12, learning_rate=0.1, subsample=1, colsample_bytree=1, eval_metric='rmse')
#reg = xgb.XGBRegressor(n_estimators=360, max_depth=3, learning_rate=0.1, subsample=1, colsample_bytree=1, eval_metric='rmse')

reg.fit(X_train, Y_train)
pred_cnt = reg.predict(X_test)


In [33]:
result = pd.DataFrame({
    "ID": test["ID"],
    "item_cnt_month": pred_cnt.clip(0. ,20.)
})
result.to_csv("submission.csv", index=False)

In [34]:
print(len(pred_cnt[pred_cnt > 20]))
result.head(30)

414


,ID,item_cnt_month
0,0,1.481584
1,1,0.045655
2,2,1.939288
3,3,0.964112
4,4,0.045655
5,5,1.883346
6,6,2.278524
7,7,2.126891
8,8,15.584173
9,9,0.045655


In [35]:
result.to_csv("submission.csv", index=False)